<a href="https://colab.research.google.com/github/WilliamQin65/cap-comp215/blob/main/SIR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

class SIR_CA:
    def __init__(self, width, height, transmission_rate, recovery_rate, initial_state):
        self.width = width
        self.height = height
        self.transmission_rate = transmission_rate
        self.recovery_rate = recovery_rate
        self.grid = np.array(initial_state)
        self.time_step = 0
        self.num_infected_history = []
        self.num_susceptible_history = []
        self.num_recovered_history = []

    def step(self, vaccination_rate=0):
        # Calculate the number of infected neighbors for each cell
        neighbor_kernel = np.array([[0, 1, 0],
                                    [1, 0, 1],
                                    [0, 1, 0]])
        num_infected_neighbors = np.zeros((self.width, self.height), dtype=int)
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                if dx == 0 and dy == 0:
                    continue
                shifted_grid = np.roll(self.grid, (dx, dy), axis=(0, 1))
                num_infected_neighbors += neighbor_kernel * (shifted_grid == 1)

        # Update the grid based on the SIR model
        new_grid = np.copy(self.grid)
        susceptible_cells = (self.grid == 0)
        infected_cells = (self.grid == 1)
        recovered_cells = (self.grid == 2)
        new_grid[susceptible_cells] = np.random.binomial(1, (1 - vaccination_rate) * (1 - (1 - self.transmission_rate) ** num_infected_neighbors[susceptible_cells]))
        new_grid[infected_cells] = np.random.binomial(1, 1 - self.recovery_rate)
        new_grid[recovered_cells] = 2
        self.grid = new_grid

        # Update the number of cells in each state
        self.num_infected_history.append(np.sum(self.grid == 1))
        self.num_susceptible_history.append(np.sum(self.grid == 0))
        self.num_recovered_history.append(np.sum(self.grid == 2))

        self.time_step += 1

    def run(self, num_steps, vaccination_rate=0):
        for i in range(num_steps):
            self.step(vaccination_rate)

    def animate(self, num_steps, interval, vaccination_rate=0):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        ims = []
        for i in range(num_steps):
            im = ax1.imshow(self.grid, cmap='viridis', animated=True)
            ax1.set_title(f"Time Step: {i}")
            ims.append([im])
            self.step(vaccination_rate)
        ani = animation.ArtistAnimation(fig, ims, interval=interval, blit=True, repeat_delay=1000)